# Hands On Tutorial 1: Intro to Keras

### Imports and parameters

In [ ]:
import keras
import keras.backend as K
from keras.datasets import mnist, imdb # Keras has a nice dataset API to download a set of curated datasets
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
batch_size = 128  # The number of examples to accumulate for a single gradient update in SGD.
num_classes = 10  # The number of classes equals the number of digits.
epochs = 50  # The number of epochs to train the model for.

### Dataset preparation

In [ ]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
DPI = 100
zoom = 50
shape= (1, 4*7*4*7)
fig, ax = plt.subplots(1, 1, figsize=(zoom*shape[1]/DPI, zoom*shape[0]/DPI), dpi=DPI)
ax.imshow(x_test[0].reshape(shape), interpolation=None)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.show()

### Building Model Architecture

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

### Compile model and set loss function and metrics

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

### Model Training

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

### Model Evaluation

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.plot(range(1, epochs+1), history.history['acc'], label='Training')
ax1.plot(range(1, epochs+1), history.history['val_acc'], label='Validation')
ax1.legend(loc='lower right')
ax1.set_xlim(1, epochs)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Categorical Accuracy')
ax2.plot(range(1, epochs+1), history.history['loss'], label='Training')
ax2.plot(range(1, epochs+1), history.history['val_loss'], label='Validation')
ax2.legend(loc='upper right')
ax2.set_xlim(1, epochs)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Categorical Crossentropy')
plt.show()

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Best validation loss: {0:.6f}'.format(score[0]))
print('Best validation accuracy: {0:.2f}%'.format(score[1]*100))

### Better model: Convolutional Neural Network

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten

In [ ]:
img_rows, img_cols = 28, 28
epochs = 20
    
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(num_classes, activation='softmax'))

cnn_model.summary()

In [ ]:
cnn_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=SGD(),
              metrics=['accuracy'])

cnn_history = cnn_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.plot(range(1, epochs+1), cnn_history.history['acc'], label='Training')
ax1.plot(range(1, epochs+1), cnn_history.history['val_acc'], label='Validation')
ax1.legend(loc='lower right')
ax1.set_xlim(1, epochs)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Categorical Accuracy')
ax2.plot(range(1, epochs+1), cnn_history.history['loss'], label='Training')
ax2.plot(range(1, epochs+1), cnn_history.history['val_loss'], label='Validation')
ax2.legend(loc='upper right')
ax2.set_xlim(1, epochs)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Categorical Crossentropy')
plt.show()

In [ ]:
score = cnn_model.evaluate(x_test, y_test, verbose=0)
print('Best validation loss: {0:.6f}'.format(score[0]))
print('Best validation accuracy: {0:.2f}%'.format(score[1]*100))